In [ ]:
NUM_COLUMN = 5
NUM_ROW = 5
MAX_PROB = 8
# Initialize game state
movesByTime = []
movesByColumn = [[] for i in range(NUM_COLUMN)]
fullColumns = [0 for i in range(NUM_COLUMN)]
time = 0
curPlayer = 0

In [ ]:
class Move:
    def __init__(self, i, c1, c2, p1, p2, pl):
        self.index = i
        # Ensure Column 1 is "dominant" in probability; in particular, ensure if
        # a probability is 0 then it's Prob 2
        if p1 < p2:
            c1, c2 = c2, c1
            p1, p2 = p2, p1
        self.col1 = c1
        self.col2 = c2
        self.prob1 = p1
        self.prob2 = p2
        self.player = pl

In [ ]:
def checkMeasure(col):
    if len(movesByColumn[col]) == NUM_ROW:
        print("Column " + col + " may be full, measuring...\n")
        measure(col)
        print("After measuring, here is the board:\n")
        displayBoard()
        if fullColumns[col]:
            print("Column " + col + " is full!")
            return True
    return False

In [ ]:
def getMove(pl):
    global time
    # Display valid move inputs (which columns are valid? How to input probabilities? What to do if they don't want
    # to use a superposition?)

    legalMoves = []
    for i in range(NUM_COLUMN):
        if i not in fullColumns:
            legalMoves.append(str(i))

    # Prompt for primary column choice
    print("Possible moves:\n")
    print(legalMoves)
    print("Choose a column:\n")
    c1 = input().strip()
    while c1 not in legalMoves:
        print("Invalid input. Try again.\n")
        print("Possible moves:\n")
        print(legalMoves)
        print("Choose a column:\n")
        c1 = input().strip()
    legalMoves.remove(c1)
    c1 = int(c1)

    # prompt for option to superimpose another column
    print("Do you want to superimpose another column? (Y/N):\n")
    opt = input().strip().upper()
    while (opt != 'Y') and (opt != 'N'):
        print("Invalid input. Try again.\n")
        print("Do you want to superimpose another column? (Y/N):\n")
        opt = input().strip().upper()

    if opt == 'Y':
        # prompt for secondary column choice
        print("Possible moves:\n")
        print(legalMoves)
        print("Choose a second column:\n")
        c2 = input().strip()
        while c2 not in legalMoves:
            print("Invalid input. Try again.\n")
            print("Possible moves:\n")
            print(legalMoves)
            print("Choose a second column:\n")
            c2 = input().strip()
        c2 = int(c2)

        # prompts probability
        options = range(1, MAX_PROB)
        legalProbs = ""
        for i in options:
            legalProbs = legalProbs + f"({i}): {i}/{MAX_PROB}\n"
        print("Enter a probability for the second column:\n")
        print(f"Choose one: \n{legalProbs} \n")
        p2 = int(input().strip())
        while p2 not in options:
            print("Invalid input. Try again.\n")
            print("Enter a probability for the second column:\n")
            print(f"Choose one: {legalProbs} \n")
            p2 = int(input().strip())
        p1 = 8 - p2
    else:
        # when no superposition occurs
        c2 = c1
        p1 = MAX_PROB
        p2 = 0

    newMove = Move(time, c1, c2, p1, p2, pl)
    # if c1 and c2 are full, a piece is not played
    isFull_c1 = checkMeasure(c1)
    isFull_c2 = checkMeasure(c2)
    if isFull_c1 & isFull_c2:
        print("Both columns are full! No piece is played. Next player's turn.")
        return
    if isFull_c1:
        # newMove becomes 100% c2
        newMove.col1 = c2
        newMove.col2 = c2
        newMove.prob1 = MAX_PROB
        newMove.prob2 = 0
    if isFull_c2:
        # newMove becomes 100% c1
        newMove.col2 = c1
        newMove.prob1 = MAX_PROB
        newMove.prob2 = 0

    # add to movesByTime
    movesByTime.append(newMove)
    time += 1
    # add to movesByColumn
    movesByColumn[c1].append(newMove.index)
    if newMove.prob2 > 0:
        movesByColumn[c2].append(newMove.index)
